In [2]:
# Run this first!!!
from IPython.display import display, HTML

from __future__ import division
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('..')
import util
sys.path.append('../common')
import sampling_misc

In [21]:
def get_theoretical(probs, m):
    return _expand_possibilities(probs, m)

def _expand_possibilities(probs, m, curr=None, index=0):
    if curr is None:
        curr = [False for _ in range(len(probs))]
    if index == len(probs):
        return _get_theor_term(probs, m, curr)
    nxt = list(curr)
    nxt[index] = True
    return (_expand_possibilities(probs, m, nxt, index + 1)
            + _expand_possibilities(probs, m, curr, index + 1))

def _get_theor_term(probs, m, subset):
    if True not in subset:
        return 0
    prob_f = 1
    prob_x = 1
    for i, include in enumerate(subset):
        p = probs[i]
        if include:
            prob_f *= p
            prob_x *= (1 - (1 - p) ** (m - 1) * p)
        else:
            prob_f *= (1 - p)
    return (1 - prob_x) * prob_f

def compare(probs, m, num_samples, depth):
    theor = get_theoretical(probs, m)
    df = sampling_misc.sample_state_has_new_thread(num_samples, probs, depth, [m])
    sampled = df['has_new_thread'].mean()
    print 'Theoretical: %f' % theor
    print 'Sampled: %f' % sampled
    print 'Relative Error: %f' % (abs(sampled - theor) / theor)

In [25]:
# PARAMETERS
PROBS = [0.5 for _ in range(4)] # Probability of seeing each letter.
NUM_SAMPLES = 1000
DEPTH = 10 # Depth to check state at
M = 2 # Size of parent

compare(PROBS, M, NUM_SAMPLES, DEPTH)

Theoretical: 0.413818
Sampled: 0.498858
Relative Error: 0.205501
